# Step 0.1. Import necessary libraries 

In [1]:
# Standard python libraries
import logging
import os
import time

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
import torch
from matplotlib import pyplot as plt

# Imports from our package
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBSeqSimpleFeatures, LGBMultiSeqSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.reader.base import PandasToPandasReader, DictToNumpySeqReader
from lightautoml.tasks import Task
from lightautoml.automl.blend import WeightedBlender
from lightautoml.dataset.seq_np_pd_dataset import SeqNumpyPandasDataset
from lightautoml.dataset.roles import NumericRole, CategoryRole, DatetimeRole
from lightautoml.dataset.np_pd_dataset import NumpyDataset, PandasDataset
from lightautoml.ml_algo.random_forest import RandomForestSklearn
from lightautoml.validation.np_iterators import TimeSeriesIterator

In [2]:
DATA_FOLDER = '/home/simakovde/hdd/kaggle/idao21_final/idao_2021_train'
train = pd.read_csv(f'{DATA_FOLDER}/funnel.csv' )

clients = pd.read_csv(f'{DATA_FOLDER}/client.csv')
deals = pd.read_csv(f'{DATA_FOLDER}/deals.csv')
aum = pd.read_csv(f'{DATA_FOLDER}/aum.csv')
trxn = pd.read_csv(f'{DATA_FOLDER}/trxn.csv')
com = pd.read_csv(f'{DATA_FOLDER}/com.csv')
appl = pd.read_csv(f'{DATA_FOLDER}/appl.csv')
payments = pd.read_csv(f'{DATA_FOLDER}/payments.csv')
balance = pd.read_csv(f'{DATA_FOLDER}/balance.csv')

W0907 22:45:24.492595 140242079766336 warnings.py:110] /home/simakovde/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.




In [3]:
train.shape, clients.shape, deals.shape, aum.shape, trxn.shape, com.shape, appl.shape, payments.shape, balance.shape

((21498, 16),
 (21498, 8),
 (109016, 8),
 (117392, 4),
 (3035705, 11),
 (113055, 10),
 (12030, 6),
 (188068, 4),
 (1194684, 9))

In [4]:
train.head()

,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,region_cd
0,7513301859607023584,0,NaN,1,7,13.0,571533.0,15717.0,0.0,0.0,0.0,571852.0,472605.0,10.4,12548.0,86.0
1,9157009756404187626,0,NaN,1,3,13.0,3642369.0,94787.0,0.0,0.0,84823.0,3642369.0,3314257.0,8.9,77210.0,2.0
2,-1893104556496814867,0,NaN,1,5,16.0,352826.0,5500.0,0.0,6822.0,0.0,265893.0,204534.0,8.9,5508.0,52.0
3,6886062013213911831,0,NaN,1,4,3.0,6070615.0,40580.0,0.0,30401.0,0.0,2005731.0,1825051.0,7.9,40583.0,86.0
4,-8156468515495593794,1,138018.05,1,7,14.0,3642369.0,97156.0,81488.0,0.0,160308.0,3642369.0,3314257.0,10.4,78108.0,27.0


In [5]:
clients.head()

,client_id,gender,age,region,city,citizenship,education,job_type
0,7513301859607023584,F,33.0,0,115,RUSSIA,NaN,NaN
1,9157009756404187626,F,59.0,17,668,RUSSIA,NaN,NaN
2,-1893104556496814867,M,51.0,28,65,RUSSIA,NaN,NaN
3,6886062013213911831,F,56.0,0,40,RUSSIA,NaN,NaN
4,-8156468515495593794,F,34.0,-1,-1,RUSSIA,HIGHER_PROFESSIONAL,NaN


In [6]:
balance.head()

,client_id,crncy_cd,eop_bal_sum_rur,min_bal_sum_rur,max_bal_sum_rur,avg_bal_sum_rur,month_end_dt,prod_cat_name,prod_group_name
0,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
1,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
2,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
3,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
4,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand


In [7]:
# в доп таблицах только данные текущих актуальных клиентов <- реализован
# в доп таблицах данные для всех клиентов
# в доп таблицах данные только для новых клиентов 

In [8]:
seq_params = {'clients': {'case': 'ids',
                          'params': {},
                          'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                         }
             }

X_train = {'plain':train , 'seq': None}
task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=4)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)


I0907 22:45:29.067527 140242079766336 utils.py:129] Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
I0907 22:45:29.069869 140242079766336 utils.py:141] NumExpr defaulting to 8 threads.


[22:45:29] Feats was rejected during automatic roles guess: []
[22:45:29] Layer 1 train process start. Time left 9999999996.50 secs
[22:45:29] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[22:45:29] Training params: {'task': 'train', 'learning_rate': 0.05, 'num_leaves': 128, 'feature_fraction': 0.7, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'max_depth': -1, 'verbosity': -1, 'reg_alpha': 1, 'reg_lambda': 0.0, 'min_split_gain': 0.0, 'zero_as_missing': False, 'num_threads': 4, 'max_bin': 255, 'min_data_in_bin': 3, 'num_trees': 3000, 'early_stopping_rounds': 100, 'random_state': 42}
[22:45:29] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:45:29] Training until validation scores don't improve for 100 rounds
[22:45:29] [100]	valid's binary_logloss: 0.354864
[22:45:30] Early stopping, best iteration is:
[63]	valid's binary_logloss: 0.351069
[22:45:30] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:45:30] Training until validation score

In [9]:
%%time
seq_params = {'trxn': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       }
             }

X_train = {'plain':train , 'seq': {'trxn': trxn}}

task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)


[22:45:34] Feats was rejected during automatic roles guess: []
[22:45:34] Layer 1 train process start. Time left 9999999997.11 secs
[22:45:45] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[22:45:45] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:45:46] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:45:47] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:45:48] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = -0.3468762868274541
[22:45:48] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[22:45:48] Time left 9999999984.05 secs

[22:45:48] Layer 1 training completed.

log-loss: 0.3468762868274541, roc_auc: 0.8231983444548147
CPU times: user 32.8 s, sys: 1.81 s, total: 34.6 s
Wall time: 27.1 s


In [10]:
%%time
seq_params = {'clients': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id', 'type': 'lookup'}
                         }
             }

X_train = {'plain':train , 'seq': {'clients': clients}}
task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)


[22:46:00] Feats was rejected during automatic roles guess: []
[22:46:00] Layer 1 train process start. Time left 9999999999.03 secs
[22:46:01] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[22:46:01] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:46:02] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:46:03] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:46:04] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = -0.24634341369370094
[22:46:04] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[22:46:04] Time left 9999999994.51 secs

[22:46:04] Layer 1 training completed.

log-loss: 0.24634341369370094, roc_auc: 0.9212829858762476
CPU times: user 17 s, sys: 292 ms, total: 17.3 s
Wall time: 7.38 s


In [11]:
%%time
seq_params = {'balance': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       }, 
              'payments': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       }, 
              'appl': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'com': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'trxn': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'aum': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'deals': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'clients': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id', 'type': 'lookup'}
                       }
             }
X_train = {'plain':train ,
           'seq': {'balance': balance,
                   'payments': payments,
                   'appl': appl,
                   'com': com,
                   'trxn': trxn,
                   'aum': aum,
                   'deals': deals,
                   'clients': clients}}

task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)

[22:46:13] Feats was rejected during automatic roles guess: []
[22:46:13] Layer 1 train process start. Time left 9999999992.98 secs
[22:46:37] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[22:46:37] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:46:38] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:46:40] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[22:46:41] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = -0.2256407674203354
[22:46:41] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[22:46:41] Time left 9999999965.33 secs

[22:46:41] Layer 1 training completed.

log-loss: 0.2256407674203354, roc_auc: 0.9368633017766725
CPU times: user 1min 11s, sys: 2.08 s, total: 1min 13s
Wall time: 1min 2s
